# DNLI Finetuning

In [1]:
#Pray for P100
!nvidia-smi

Thu Nov 28 20:06:57 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    33W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
#Get DNLI dataset
!pip install gdown
!gdown --id 1WtbXCv3vPB5ql6w0FVDmAEMmWadbrCuG --output dnli.zip
!unzip dnli.zip

Downloading...
From: https://drive.google.com/uc?id=1WtbXCv3vPB5ql6w0FVDmAEMmWadbrCuG
To: /content/dnli.zip
390MB [00:03, 104MB/s]
Archive:  dnli.zip
   creating: dnli/
   creating: dnli/dialogue_nli/
  inflating: dnli/dialogue_nli/dialogue_nli_train.jsonl  
  inflating: dnli/dialogue_nli/print_dtypes.py  
  inflating: dnli/dialogue_nli/README.txt  
   creating: __MACOSX/
   creating: __MACOSX/dnli/
   creating: __MACOSX/dnli/dialogue_nli/
  inflating: __MACOSX/dnli/dialogue_nli/._README.txt  
  inflating: dnli/dialogue_nli/dialogue_nli_dev.jsonl  
  inflating: __MACOSX/dnli/dialogue_nli/._dialogue_nli_dev.jsonl  
  inflating: dnli/dialogue_nli/dialogue_nli_test.jsonl  
  inflating: dnli/dialogue_nli/dialogue_nli_verified_test.jsonl  
  inflating: dnli/.DS_Store          
  inflating: __MACOSX/dnli/._.DS_Store  
  inflating: dnli/dialogue_nli.tar.gz  
   creating: dnli/dialogue_nli_extra/
  inflating: dnli/dialogue_nli_extra/dialogue_nli_EXTRA_uu_test.jsonl  
  inflating: dnli/dialogue

In [0]:
# Prepare tsv files as required by DataProcessor class

import json
import pandas as pd
import csv

#Read jsonl files for train/val
with open('dnli/dialogue_nli/dialogue_nli_train.jsonl') as f:
    lines = f.readlines()
train_data = json.loads(lines[0])
with open('dnli/dialogue_nli/dialogue_nli_dev.jsonl') as f:
    lines = f.readlines()
val_data = json.loads(lines[0])

#Make dataframes
df_train = pd.DataFrame.from_dict(train_data, orient='columns')
df_train['index'] = df_train.index
df_train = df_train[['index','sentence1', 'sentence2','label']]
df_train['label'] = df_train['label'].replace(['negative', 'positive'], ['contradiction', 'entailment'])

df_val = pd.DataFrame.from_dict(val_data, orient='columns')
df_val['index'] = df_val.index
df_val = df_val[['index','sentence1', 'sentence2','label']]
df_val['label'] = df_val['label'].replace(['negative', 'positive'], ['contradiction', 'entailment'])

!mkdir dnli_data
df_train.to_csv('dnli_data/train.tsv', sep='\t',quoting=csv.QUOTE_NONE, index=False)
df_val.to_csv('dnli_data/dev.tsv', sep='\t',quoting=csv.QUOTE_NONE, index=False)

In [4]:
!git clone https://github.com/echodarkstar/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 12711, done.
remote: Total 12711 (delta 0), reused 0 (delta 0), pack-reused 12711
Receiving objects: 100% (12711/12711), 6.58 MiB | 11.95 MiB/s, done.
Resolving deltas: 100% (9287/9287), done.


In [5]:
!cd transformers && python setup.py develop

running develop
running egg_info
creating transformers.egg-info
writing transformers.egg-info/PKG-INFO
writing dependency_links to transformers.egg-info/dependency_links.txt
writing entry points to transformers.egg-info/entry_points.txt
writing requirements to transformers.egg-info/requires.txt
writing top-level names to transformers.egg-info/top_level.txt
writing manifest file 'transformers.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'transformers.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.6/dist-packages/transformers.egg-link (link to .)
Adding transformers 2.2.0 to easy-install.pth file
Installing transformers script to /usr/local/bin

Installed /content/transformers
Processing dependencies for transformers==2.2.0
Searching for sacremoses
Reading https://pypi.org/simple/sacremoses/
Best match: sacremoses 0.0.35
Processing sacremoses-0.0.35.tar.gz
Writing /tmp/easy_install-swst0yaj/sacremoses-0.0.35/setup.cfg
Runnin

In [0]:
#Fine tuning is crucuial for us. because the dataset is very similar: persona based dialogue exchanges
#/ inference format
#/ not sentence based inference, but UTTERANCE based inference. one utterance can consist of
# multiple small sentences (usually max 2)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
#Fine tuning command. 
#--model_name_or_path accepts pretrained file path (or a name in huggingface model zoo
#Takes 4.5 hours to train on P100
!python transformers/examples/run_glue.py \
    --model_type roberta \
    --model_name_or_path "drive/My Drive/NLP/project/models/roberta-dnli" \
    --task_name dnli \
    --do_train\
    --do_eval \
    --do_lower_case \
    --data_dir dnli_data \
    --max_seq_length 128 \
    --per_gpu_train_batch_size 16 \
    --learning_rate 2e-5 \
    --save_steps 5000\
    --num_train_epochs 1 \
    --output_dir "drive/My Drive/NLP/project/models/roberta-dnli-v2"

11/28/2019 08:07:19 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/28/2019 08:07:19 - INFO - transformers.configuration_utils -   loading configuration file drive/My Drive/NLP/project/models/roberta-dnli/config.json
11/28/2019 08:07:19 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "dnli",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 3,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 1,
  "use_bfloat16": false,
  "vocab_size": 50265
}

11/28/2019 08:07:19 - INFO - transformers.tokenization_util

# GLUE training

In [0]:
!git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git

Cloning into '60c2bdb54d156a41194446737ce03e2e'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21
Unpacking objects: 100% (21/21), done.


In [0]:
!mv /content/60c2bdb54d156a41194446737ce03e2e/download_glue_data.py download_glue_data.py

In [0]:
!python download_glue_data.py --data_dir glue_data --tasks all

	Completed!
	Completed!
Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!


In [0]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_glue.py

--2019-11-26 20:22:53--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/run_glue.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28343 (28K) [text/plain]
Saving to: ‘run_glue.py’

run_glue.py         100%[===================>]  27.68K  --.-KB/s    in 0.007s  

2019-11-26 20:22:54 (4.02 MB/s) - ‘run_glue.py’ saved [28343/28343]



In [0]:
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=caf49603d155cd244e405ad1f732d2bfd1eff3f979686ae6f02487e92e95b379
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [0]:
!python run_glue.py \
    --model_type roberta \
    --model_name_or_path roberta-large-mnli \
    --task_name mnli \
    --do_train \
    --do_eval \
    --do_lower_case \
    --data_dir dnli_data \
    --max_seq_length 128 \
    --per_gpu_train_batch_size 8 \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --output_dir output_dir \
    --overwrite_output_dir

11/26/2019 23:11:32 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
11/26/2019 23:11:32 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-mnli-config.json from cache at /root/.cache/torch/transformers/54eef9bf74f919edd81b765fee413c8229620f3e271a51bdcdc67797422ef3f3.70abe4626d3e230d17c121572ba4568ae897a527d380497381d297d714878e9f
11/26/2019 23:11:32 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "mnli",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 3,
  "output_attentions": false,
  "output_hidden_states": false,
  